In [1]:
src = "/user/student/authors/"
authorNameRDD = sc.textFile(src)\
    .map(lambda line: (int(line.split(",")[0]),(line.split(",")[1],line.split(",")[2])))

authorNameRDD.take(5)

[(1, ('Walton', 'Adams')),
 (2, ('Marietta', 'Walsh')),
 (3, ('Lily', 'Wintheiser')),
 (4, ('Estevan', 'Gleason')),
 (5, ('Thaddeus', 'Rowe'))]

In [2]:
src = "/user/student/posts/"
postsRDD = sc.textFile(src)\
        .map(lambda line: (line.split(",")[0],
                        (line.split(",")[1],line.split(",")[2][0:10])))
postsRDD.take(5)

[('1', ('1', 'Cupiditate')),
 ('2', ('2', 'Excepturi ')),
 ('3', ('3', 'Enim rerum')),
 ('4', ('4', 'Labore ips')),
 ('5', ('5', 'Placeat ex'))]

In [4]:
import json

def setPhoneName(s):
    if s == "" :return "Unknown"
    elif "," in s: return s.replace(",", "or")
    else: return s
    
src = "/user/student/author_phone.json"
phoneRDD = sc.wholeTextFiles(src)\
        .flatMap(lambda tup: json.loads(tup[1]))\
        .map(lambda kv: (kv.get("author_id",None),kv.get("phone_model",None)))\
        .map(lambda kv: (kv[0],setPhoneName(kv[1])))


phoneRDD.take(5)

[(1, 'Samsung Galaxy s21 Ultra'),
 (2, 'Samsung Galaxy A52s'),
 (3, 'Samsung Galaxy Z Fold3'),
 (4, 'Apple iPhone 7'),
 (5, 'Apple iPhone 11')]

In [5]:
import xml.etree.ElementTree as ET
def getPosts(s):
    posts = ET.fromstring(s)
    return posts.iter("record")

def getPostID(elem):
    return elem.find("post_id").text

def getPostLocation(elem):
    return elem.find("location").text

In [12]:
latLongRDD = sc.wholeTextFiles("post_records/*")\
    .flatMap(lambda xmls: getPosts(xmls[1])) \
    .map(lambda element: (getPostID(element),tuple(getPostLocation(element).split(",") ) ) )
latLongRDD.take(5)

2024-08-30 06:59:36,418 WARN impl.BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:657)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:531)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:2939)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:821)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:746)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:379)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:644)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:575)
	

[('1', ('-78.67343', ' 146.15251')),
 ('2', ('-24.8449', ' 71.73862')),
 ('3', ('33.15167', ' 90.2584')),
 ('4', ('78.53576', ' -113.2306')),
 ('5', ('37.09904', ' 141.78509'))]

In [13]:
authorPhoneRDD = authorNameRDD.join(phoneRDD)
authorPhoneRDD.take(5)

[(5, (('Thaddeus', 'Rowe'), 'Apple iPhone 11')),
 (10, (('Dewitt', 'Smitham'), 'Unknown')),
 (15, (('Brenda', 'Mayer'), 'Apple iPhone 8')),
 (20, (('Wilfredo', 'Yundt'), 'Samsung Galaxy A12or Apple iPhone 6')),
 (25, (('Westley', 'Rempel'), 'Unknown'))]

In [8]:
authorPhoneRDD.count()

10000

In [9]:
postLocationRDD  = postsRDD.join(latLongRDD)
postLocationRDD.take(5)

[('4', (('4', 'Labore ips'), ('78.53576', ' -113.2306'))),
 ('16', (('16', 'Eius ea ha'), ('41.55404', ' 7.26911'))),
 ('20', (('20', 'Facere atq'), ('71.33242', ' 7.33176'))),
 ('22', (('22', 'Sint iusto'), ('2.78725', ' 124.55732'))),
 ('24', (('24', 'Dolorem ve'), ('56.27864', ' -115.38594')))]

In [10]:
postLocationRDD.count()

110000

In [11]:
authorPostLocationRDD = postLocationRDD\
.map(lambda tup:(int(tup[1][0][0]),[tup[1][0][1],tup[1][1][0],tup[1][1][1]] ) )
authorPostLocationRDD.take(5)

[(4, ['Labore ips', '78.53576', ' -113.2306']),
 (16, ['Eius ea ha', '41.55404', ' 7.26911']),
 (20, ['Facere atq', '71.33242', ' 7.33176']),
 (22, ['Sint iusto', '2.78725', ' 124.55732']),
 (24, ['Dolorem ve', '56.27864', ' -115.38594'])]

In [14]:
authorNamePhoneRDD = authorPhoneRDD\
.map(lambda tup: (tup[0],[tup[1][0][0],tup[1][0][1],tup[1][1]]) )

authorNamePhoneRDD.take(5)

[(5, ['Thaddeus', 'Rowe', 'Apple iPhone 11']),
 (10, ['Dewitt', 'Smitham', 'Unknown']),
 (15, ['Brenda', 'Mayer', 'Apple iPhone 8']),
 (20, ['Wilfredo', 'Yundt', 'Samsung Galaxy A12or Apple iPhone 6']),
 (25, ['Westley', 'Rempel', 'Unknown'])]

In [15]:
nameTitlePhoneLocRDD = authorNamePhoneRDD.join(authorPostLocationRDD)\
.map(lambda tup: 
     (tup[1][0][0]+" "+tup[1][0][1]+" on "+tup[1][0][2],
      "Posted "+tup[1][1][0]+" from lat: "+tup[1][1][1]+" lon:"+tup[1][1][2] ) )
nameTitlePhoneLocRDD.take(5)

[('Brigitte Shanahan on Unknown',
  'Posted Et sit har from lat: 70.80241 lon: 32.37947'),
 ('Brigitte Shanahan on Unknown',
  'Posted Voluptates from lat: -64.13821 lon: -129.31313'),
 ('Brigitte Shanahan on Unknown',
  'Posted Vel debiti from lat: -15.29791 lon: -110.17868'),
 ('Brigitte Shanahan on Unknown',
  'Posted Debitis am from lat: 30.79221 lon: 73.33461'),
 ('Brigitte Shanahan on Unknown',
  'Posted Possimus n from lat: 56.51724 lon: 19.96479')]

In [16]:
ZeroValue =[]

def seqOp(accumulator,element):
    accumulator.append(element)
    return accumulator

def combOp(accumulator1,accumulator2):
    return accumulator1 + accumulator2

finalRDD = nameTitlePhoneLocRDD.aggregateByKey(ZeroValue,seqOp,combOp)

finalRDD.take(5)


[('Laurianne Jerde on Samsung Galaxy S21',
  ['Posted Id repudia from lat: -70.76073 lon: 150.41541',
   'Posted Eos nihil  from lat: -60.27312 lon: 0.28329',
   'Posted Neque nam  from lat: -15.1393 lon: -153.41288',
   'Posted Quam exped from lat: 59.64538 lon: 162.21816',
   'Posted Vero enim  from lat: 75.63085 lon: -179.84681',
   'Posted Explicabo  from lat: 28.53925 lon: 39.01707',
   'Posted Quis corru from lat: 18.68474 lon: -134.40318',
   'Posted Repellat i from lat: -21.51617 lon: -178.06651',
   'Posted Magni blan from lat: -63.58662 lon: -126.2323',
   'Posted Qui non as from lat: 59.41561 lon: 140.52858',
   'Posted Est at exc from lat: -19.77205 lon: -22.35178']),
 ('Theodora Nicolas on Unknown',
  ['Posted Id odio di from lat: 60.95745 lon: 165.47261',
   'Posted Vel consec from lat: -0.87067 lon: 98.3268',
   'Posted Ea ea corr from lat: 24.36141 lon: -16.24',
   'Posted Dolor dolo from lat: -10.56048 lon: -177.68041',
   'Posted Delectus c from lat: -19.3932 lon: -11